# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 02.10.24 23:59__

__Жесткий дедлайн: 05.10.24 23:59__


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов и еще __2__ балла можно будет получить за демонстрацию вашей системы и пользовательского интерфейса. Демонстрацию прикрепляйте в anytask в виде 1-2 минутной записи экрана.

In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

%load_ext autoreload
%autoreload 2

from utils import preprocess_msg, word_piece_tokenize
from prefix_tree import PrefixTree, PrefixTreeNode
from word_completor import WordCompletor
from ngram_lm import NGramLanguageModel
from suggester import TextSuggestion, WordPieceTextSuggestion
from model import SuggestionModel

import nltk
import pandas as pd
import pickle
from typing import List, Tuple, Dict, Set, Optional, Any, Union
from tokenizers import BertWordPieceTokenizer, Tokenizer

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

# ALERT! не рекомендую запускать больше чем на 100k - процессор сдохнет. самая жирная модель училась на 75к
emails = pd.read_csv("emails.csv", nrows=15000)
len(emails)

75000

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [32]:
emails["clean_message"] = emails["message"].apply(preprocess_msg)

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [5]:
# беру пути из дз по мо-2

if "/Users/andreypetukhov/Documents/Машинное-обучение/ML1-and-ML2/homework-practice-10-unsupervised/nltk_data" not in nltk.data.path:
    nltk.data.path.append("/Users/andreypetukhov/Documents/Машинное-обучение/ML1-and-ML2/homework-practice-10-unsupervised/nltk_data")

if "/Users/andreypetukhov/Documents/Машинное-обучение/ML1-and-ML2/homework-practice-10-unsupervised/corpora" not in nltk.data.path:
    nltk.data.path.append("/Users/andreypetukhov/Documents/Машинное-обучение/ML1-and-ML2/homework-practice-10-unsupervised/corpora")

In [ ]:
# токенизирую 

emails["word_tokens"] = emails["clean_message"].apply(nltk.word_tokenize)

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [6]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [7]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [8]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [35]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [36]:
word_completor = WordCompletor(emails["word_tokens"])
n_gram_model = NGramLanguageModel(corpus=emails["word_tokens"], n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

In [37]:
text_suggestion.suggest_text(["i", 'have', 'been', 'working'], n_words=3, n_texts=1)

[['working', 'with', 'the', 'best']]

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!

Всё оформлено в питоновских файлах, описано в отчёте. Ниже демонстрация работы улучшенного класса с бим-сёрчем.

In [33]:
# заводим умный токенизатор

tokenizer = BertWordPieceTokenizer()
tokenizer.enable_padding()
tokenizer.train_from_iterator(emails["clean_message"])

In [26]:
# нашим токенизатором обрабатываем чистые сообщения

emails["wordpiece_tokens"] = emails["clean_message"].apply(lambda msg: word_piece_tokenize(msg, tokenizer))

# заводим модель

model = SuggestionModel(corpus=emails["wordpiece_tokens"], ngram_order=3, tokenizer=tokenizer)

Обучился дополнятель слов
Обучилась n-gram модель
Инициализировали токенайзер


In [34]:
# нашим токенизатором обрабатываем чистые сообщения

emails["wordpiece_tokens"] = emails["clean_message"].apply(lambda msg: word_piece_tokenize(msg, tokenizer))

# заводим модель

model = SuggestionModel(corpus=emails["wordpiece_tokens"], ngram_order=3, tokenizer=tokenizer)

Обучился дополнятель слов
Обучилась n-gram модель
Инициализировали токенайзер


In [20]:
# пробуем её

model.predict_sentence("what woul", n_words=3, n_texts=5)

['would youexpect somebody',
 'would you like to',
 'would you want to',
 'would you do?',
 'would you do me']

Видим отличные предложения - слово дополнило шикарно и предложение тоже! И работает молниеносно! Сохраним модели:

In [13]:
with open('wc/wc_75k_3gram_2.pkl', 'wb') as wc_file:
    pickle.dump(model.word_completor, wc_file)

with open('ng/ng_75k_3gram_2.pkl', 'wb') as ngram_file:
    pickle.dump(model.n_gram_model, ngram_file)

tokenizer.save("tokenizers/tokenizer_75k_2.json")

Если захочется загрузить модель из файлов, то делать это стоит таким образом:

In [65]:
with open('wc/wc_75k_3gram.pkl', 'rb') as wc_file:
    wc = pickle.load(wc_file)

with open('ng/ng_75k_3gram.pkl', 'rb') as ngram_file:
    ng = pickle.load(ngram_file)

tokenizer = Tokenizer.from_file("tokenizers/tokenizer_75k.json")

model = SuggestionModel(corpus=[[]], ngram_order=3, tokenizer=tokenizer)
model.word_completor = wc
model.n_gram_model = ng
model.suggester = WordPieceTextSuggestion(word_completor=wc, n_gram_model=ng, tokenizer=tokenizer)

Обучился дополнятель слов
Обучилась n-gram модель
Инициализировали токенайзер


In [64]:
model.predict_sentence("oh god... it must be", n_texts=3, n_words=3)

['be sent by you', 'be sent by the', 'be sent by federal']